In [4]:
from itertools import product, islice
from typing import Dict, List, Any, Tuple, Callable
import os
import pickle
from timeit import default_timer as timer
import math

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import ConvexHull, Delaunay
from scipy.spatial import distance

from IOData.IOData import InputRule
from IOData.IODataWith_l import IODataWith_l

from simulators.track_simulator import TrackSimulator
from simulators.simulation_settings import SafetyFilterTypes, TrackFilterTypes, SimulationInputRule, ModelType
from tools.simualtion_results import Results

from tools.simple_track_generator import trackGenerator
from tools.FractalDimension import fractal_dimension
from tools.monte_carlo_integral import monte_carlo_integrate
from tools.dataset_analyse import weighting_xi_in_datasets, weighting_u_pf_y_p_in_datasets, get_datasets_hankel_matrix, Sampler

In [5]:
# read results from file
results_file_name = '09-19_14_40-results_list.pkl'
with open(os.path.join(os.curdir, 'datasets', results_file_name), 'rb') as f:
    results_list: List[Results] = pickle.load(f)

In [6]:
dataset_name_list: List[str]  = [
    '09-19_14_11-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_12-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_13-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_14-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_16-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_17-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_19-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_21-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_22-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_24-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_26-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_28-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_30-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_32-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
    '09-19_14_36-oval_track-INDIRECT_FIX_MU_WEIGHTING_ADD_DATA.pkl',
]

In [15]:
for result in results_list:
    print(result.mark_time_steps)

[]
[]
[143, 127, 107]
[]
[]
[931, 915, 895]
[]
[]
[947, 931, 911]
[1115, 1099, 1079]
[]
[947, 931, 911]
[]
[]
[]


In [26]:
# parameters for analysis stored in Sampler
sampler = Sampler()
sampler.lag = 1
sampler.L = 1

# results and dataset index to analyse
results_index = [2, 5, 8, 11, 14]
dataset_list: List[List[List[IODataWith_l]]] = []
for i in results_index:
    with open(os.path.join(os.curdir, 'datasets', dataset_name_list[i]), 'rb') as f:
        single_dataset_list: List[List[IODataWith_l]] = pickle.load(f)
    dataset_list.append(single_dataset_list)

# get output and input size
m = dataset_list[0][0][0]._input_data[0].shape[0]
p = dataset_list[0][0][0]._output_data[0].shape[0]

# extended state to analyse
state_index = 127
results = results_list[results_index[0]]
u_list, y_list = [], []
for i in range(state_index-sampler.lag+1, state_index+1):
    u_list.append(results._input_applied[i])
    y_list.append(results._error_trajectory[i][:p])
xi_to_analyse = np.hstack(u_list + y_list)
print(xi_to_analyse)

# index of segment to analyse
i_seg = results._

[ 9.94903432e-01 -2.80125872e-01 -7.77214914e-02 -2.54675852e+01
  2.98149533e+00 -2.44960355e-02]


In [8]:
from itertools import islice
sampler = Sampler()
for el in islice(sampler.u_pf_y_p_iterator(),5):
    print(el)

[ 3.92494127  0.34626657 -0.48989191 -0.17664771 -0.49333619  0.87891892
  2.54733764 -1.89744206]
[ 0.49295449 -0.13922798  4.13603385  0.03844976 -0.16376907  1.39981734
  1.83084388 -1.63122561]
[ 3.41563236  0.34194036  0.66344917 -0.24127359 -0.28340587  0.03503453
  0.91009074 -1.87592194]
[-1.07814328 -0.29456163  4.25823655 -0.39850516 -0.21518657  0.54588416
  3.03212887 -0.38234845]
[ 2.76353008 -0.2269794   3.36720584 -0.13031582 -0.21655426  0.31825908
  0.38442324  0.0679664 ]
